## 02 BOOK

01 튜토리얼의 분석에서 
- 특정코인에 대하여 보다 시장의 흐름에 따라 가격이 결정되는 것으로 판단
- 따라서 시장의 흐름을 분석할 수 있는 지표를 나타낼 수 있는 값을 계산해본다.

### TODO
1. 업/다운의 수를 계산해본다.

### 유틸리티 함수입니다.

In [125]:
import pandas as pd
import pyupbit
import time
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from time import sleep

#load data
def load_prev_price(tickers):
    prev = {}
    
    for coin in tickers:
        sleep(0.1)
        
        data = pyupbit.get_ohlcv(coin, count=2)
        if data is not None:
            prev[coin] = data["close"]
        else:
            print(coin)
        
    return prev

### make dataframe for operation
 과거의 데이터를 로드합니다.

In [126]:
#load information
tickers    = pyupbit.get_tickers(fiat="KRW")
prev_price = load_prev_price(tickers)
tickers    = list(prev_price.keys())

df_prev = pd.DataFrame.from_dict(prev_price, orient='index')
df_prev.tail(10)

,2021-05-03 09:00:00,2021-05-04 09:00:00
KRW-CBK,10010.0,9285.0
KRW-SAND,804.0,799.0
KRW-HUM,204.0,189.0
KRW-DOGE,539.0,696.0
KRW-STRK,80100.0,76850.0
KRW-PUNDIX,4140.0,3865.0
KRW-FLOW,38300.0,40810.0
KRW-DAWN,10060.0,9560.0
KRW-AXS,11300.0,10920.0
KRW-STX,2550.0,2720.0


현재의 데이터를 로드합니다.

In [127]:
curr_price = pyupbit.get_current_price(tickers)

df_curr = pd.DataFrame.from_dict(curr_price, orient='index', columns=["cur"])
df_price= pd.merge(df_prev, df_curr, left_index=True, right_index=True,  how="left")

df_price.head()


,2021-05-03 09:00:00,2021-05-04 09:00:00,cur
KRW-BTC,69996000.0,69148000.0,69157000.0
KRW-ETH,4226000.0,4335000.0,4337000.0
KRW-NEO,131250.0,126950.0,126800.0
KRW-MTL,5190.0,4935.0,4940.0
KRW-LTC,361200.0,396400.0,395850.0


만들어진 데이터를 이용하여 정보를 계산합니다.

In [134]:
df_price["%"] = (df_price["cur"] - df_price[df_price.columns[0]])*100/df_price[df_price.columns[0]] 
con0 = df_price[df_price.columns[0]] - df_price["cur"] < 0
con1 = df_price[df_price.columns[0]] - df_price["cur"] > 0

print(df_price.cur[con0].count(), df_price.cur[con0].count()/df_price.cur[con1].count())
df_price[con0].tail(df_price.cur[con0].count())


19 0.2345679012345679


,2021-05-03 09:00:00,2021-05-04 09:00:00,cur,%
KRW-ETH,4226000.0,4335000.0,4337000.0,2.626597
KRW-LTC,361200.0,396400.0,395850.0,9.593023
KRW-ETC,63700.0,73110.0,73200.0,14.913658
KRW-OMG,11500.0,11780.0,11790.0,2.521739
KRW-WAVES,40420.0,44740.0,44790.0,10.811479
KRW-LSK,7260.0,7415.0,7420.0,2.203857
KRW-BTG,117800.0,125350.0,125300.0,6.366723
KRW-EOS,8340.0,8600.0,8620.0,3.357314
KRW-BCH,1246500.0,1277000.0,1280500.0,2.727637
KRW-MFT,23.0,24.8,24.8,7.826087


나의 관심사 "아이오타"는 어떻게 변하고 있는지 확인해본다.

In [135]:
df_price.loc["KRW-IOTA"]

2021-05-03 09:00:00    2645.000000
2021-05-04 09:00:00    2505.000000
cur                    2505.000000
%                        -5.293006
Name: KRW-IOTA, dtype: float64